In [1]:
from build_dataset import MRILesionDatasetBuilder

2025-05-19 17:04:39.399219: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-19 17:04:39.421807: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747667079.450327 2424393 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747667079.458753 2424393 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1747667079.479932 2424393 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

# Build the dataset for the lesion inpaiting model
- Train / test split
- Filter out slices with no lesions
- Include SHIFTS and WMH2017 datasets

### VH

In [2]:
data_folder="/home/benet/data"
input_folder="VH"
output_folder="lesion2D_VH-SHIFTS-WMH2017"
folders=["train", "test"]
flair_image="flair.nii.gz"
mask_image="lesionMask.nii.gz"
slices_per_example=13
slices_step=1
start_slice=85
train_split=0.7
seed = 17844

# dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed)
# dataset_builder.build_dataset()

output_folder="lesion2D_VH-SHIFTS-WMH2017_empty_masks"
skip_empty_masks = False
dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks)
dataset_builder.build_dataset()

Total empty masks skipped: 0
In the preprocessed folder: Total examples: 741, train examples: 507 (68.42%), test examples: 234 (31.58%)
In the hole dataset: Total examples: 741, train examples: 507 (68.42%), test examples: 234 (31.58%)


### SHIFTS

In [3]:
data_folder="/home/benet/data"
input_folder="SHIFTS_preprocessedMNI" ###
output_folder="lesion2D_VH-SHIFTS-WMH2017"
folders=["dev_in", "dev_out", "eval_in", "train"] ###
flair_image="flair.nii.gz"
mask_image="lesionMask.nii.gz"
slices_per_example=13
slices_step=1
start_slice=85
seed = 17844

# dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed)
# dataset_builder.build_dataset()

output_folder="lesion2D_VH-SHIFTS-WMH2017_empty_masks"
skip_empty_masks = False
dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks)
dataset_builder.build_dataset()

Total empty masks skipped: 0
In the preprocessed folder: Total examples: 1274, train examples: 871 (68.37%), test examples: 403 (31.63%)
In the hole dataset: Total examples: 2015, train examples: 1378 (68.39%), test examples: 637 (31.61%)


### WMH2017

In [4]:
data_folder="/home/benet/data"
input_folder="WMH2017_preprocessedMNI" ###
output_folder="lesion2D_VH-SHIFTS-WMH2017"
folders=None ###
flair_image="flair.nii.gz"
mask_image="lesionMask.nii.gz"
slices_per_example=13
slices_step=1
start_slice=85
seed = 17844

# dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed)
# dataset_builder.build_dataset()

output_folder="lesion2D_VH-SHIFTS-WMH2017_empty_masks"
skip_empty_masks = False
dataset_builder = MRILesionDatasetBuilder(data_folder, input_folder, output_folder, folders, flair_image, mask_image, slices_per_example, slices_step, start_slice, train_split, seed, skip_empty_masks)
dataset_builder.build_dataset()

Total empty masks skipped: 0
In the preprocessed folder: Total examples: 780, train examples: 546 (70.00%), test examples: 234 (30.00%)
In the hole dataset: Total examples: 2795, train examples: 1924 (68.84%), test examples: 871 (31.16%)
